In [1]:
import os
import random
import json
import requests
from operator import itemgetter
from typing import Union, List
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import CallbackManager
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec, ConfigurableField
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

#custom libraries that we will use later in the app
from common.utils import (
    SQLSearchAgent, 
    GithubUpdateTool
)
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PROMPT 

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))

In [2]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000

# We can run the everything with GPT3.5, but try also GPT4 and see the difference in the quality of responses
# You will notice that GPT3.5 is not as reliable when using multiple sources.

# llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
#                       temperature=0, max_tokens=COMPLETION_TOKENS)

# Uncomment below if you want to see the answers streaming
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], temperature=0, max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager, api_version="2024-05-01-preview")
# llm = ChatOllama(model = "llama3.2:latest", max_tokens = 128000, temperature=0, callbacks=cb_manager, disable_streaming = False)

In [3]:
sql_search = SQLSearchAgent(llm=llm, k=10, callback_manager=cb_manager,
                name="sqlsearch",
                description="useful when the questions includes the term: sqlsearch",
                verbose=False)

In [4]:
github_update_tool = GithubUpdateTool(
    name="github_update",
    description="Fetches GitHub updates for the given username from the environment variable for yesterday's date",
    verbose=False
)

# Multi Agent

In [5]:
tools = [sql_search, github_update_tool]

In [6]:
agent = create_openai_tools_agent(llm, tools, CUSTOM_CHATBOT_PROMPT)

In [7]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [8]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos

In [9]:
brain_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [10]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

# random_session_id = "session428" 
# ramdom_user_id = "user325"
config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}
print(random_session_id, ramdom_user_id)

session988 user311


# Further Testing

In [11]:
github_update_tool.invoke({})

Running GithubUpdateTool
Fetching GitHub events for user: ashishkj21 on date: 2025-01-10
Fetched events: Event Type: PushEvent
Repository: ashishkj21/DailyPulseAgentic
Date: 2025-01-10 22:32:17
Commits:
  - Message: Negative first response prompt working
    URL: https://api.github.com/repos/ashishkj21/DailyPulseAgentic/commits/75314e66ddd9f5993776169ccf506f80e113e0f0
  - Message: Merge pull request #5 from ashishkj21/ImprovedPrompt

Negative first response prompt working
    URL: https://api.github.com/repos/ashishkj21/DailyPulseAgentic/commits/61b9884f4031edb54e8e1e2a108334c02b96cf5a

Event Type: PullRequestEvent
Repository: ashishkj21/DailyPulseAgentic
Date: 2025-01-10 22:32:17  
                Action: closed  
                Pull Request URL: https://github.com/ashishkj21/DailyPulseAgentic/pull/5  
                Title: Negative first response prompt working  
                Body: None  
            

Event Type: PullRequestEvent
Repository: ashishkj21/DailyPulseAgentic
Date: 2

'Event Type: PushEvent\nRepository: ashishkj21/DailyPulseAgentic\nDate: 2025-01-10 22:32:17\nCommits:\n  - Message: Negative first response prompt working\n    URL: https://api.github.com/repos/ashishkj21/DailyPulseAgentic/commits/75314e66ddd9f5993776169ccf506f80e113e0f0\n  - Message: Merge pull request #5 from ashishkj21/ImprovedPrompt\n\nNegative first response prompt working\n    URL: https://api.github.com/repos/ashishkj21/DailyPulseAgentic/commits/61b9884f4031edb54e8e1e2a108334c02b96cf5a\n\nEvent Type: PullRequestEvent\nRepository: ashishkj21/DailyPulseAgentic\nDate: 2025-01-10 22:32:17  \n                Action: closed  \n                Pull Request URL: https://github.com/ashishkj21/DailyPulseAgentic/pull/5  \n                Title: Negative first response prompt working  \n                Body: None  \n            \n\nEvent Type: PullRequestEvent\nRepository: ashishkj21/DailyPulseAgentic\nDate: 2025-01-10 22:32:05  \n                Action: opened  \n                Pull Reque

In [12]:
# def chat_with_agent():  
#     while True:  
#         question = input("Enter your question (or type 'exit' to quit): ")  
#         if question.lower() == "exit":  
#             print("Goodbye!")  
#             break  
#         response = brain_agent_executor.invoke({"question": question}, config=config)["output"]  
#         printmd(response)  
  
# # Run the chat function  
# chat_with_agent()  